# Query Subset Selection

Publication information: Martins, D. M. L., Lechtenbörger, J., Vossen, G. (2019, November). Supporting Customers with Limited Budget in Data Marketplaces. 2019 IEEE Latin American Conference on Computational Intelligence (LA-CCI). (pp. 201-206). IEEE.

***This code uses Inspyred: https://github.com/aarongarrett/inspyred***

In [ ]:
import random
from time import time, sleep
import inspyred
import pandas as pd

In [ ]:
def get_synthetic_queries(num_queries):
    """Returns a list of queries in the form of (weight, value) tuples"""
    queries = [(random.randint(10, 1000), random.choice([1, 2, 2, 4, 4, 5, 5, 5])) for x in range(num_queries)]
    return queries

In [ ]:
def get_adult_dataset_queries():
    queries = [(2970, 2), (175, 5), (250, 4), (1355, 1), (6930, 3)]
    return queries

In [ ]:
def powerset(items):
    res = [[]]
    for item in items:
        newset = [r+[item] for r in res]
        res.extend(newset)
    return res

In [ ]:
def enumerative_solver(items, capacity):
    solution = []
    best_weight = 0
    best_value = 0
    
    for item_set in powerset(items):
        set_weight = sum([e[0] for e in item_set])
        set_value = sum([e[1] for e in item_set])
        if set_value > best_value and set_weight <= capacity:
            best_value = set_value
            best_weight = set_weight
            solution = item_set
    
    best = [best_value, best_weight, solution]
    return best

In [ ]:
def greedy_solver(items, capacity):
    taken = 0.0
    selected = []
    copyitems = items[:]
    sorted(copyitems, key=lambda x: x[1], reverse=True)
    while taken < capacity:
        for i in copyitems:
            selected.append(i)
            taken += i[0]

    return selected

In [ ]:
def genetic_algorithm_solver(items, capacity):
    problem = inspyred.benchmarks.Knapsack(capacity, items, duplicates=False)
    prng = random.Random()
    prng.seed(time())
    
    ea = inspyred.ec.EvolutionaryComputation(prng)
    ea.selector = inspyred.ec.selectors.tournament_selection
    ea.variator = [inspyred.ec.variators.uniform_crossover, 
                inspyred.ec.variators.gaussian_mutation]
    ea.replacer = inspyred.ec.replacers.steady_state_replacement
    #ea.terminator = inspyred.ec.terminators.evaluation_termination
    ea.terminator = inspyred.ec.terminators.generation_termination

    final_pop = ea.evolve(generator=problem.generator, 
                        evaluator=problem.evaluator, 
                        bounder=problem.bounder,
                        maximize=problem.maximize, 
                        pop_size=100, 
                        max_evaluations=3000,
                        max_generations=max(10, MAXGEN),
                        tournament_size=5,
                        num_selected=10,
                        mutation_rate=0.20,
                        crossover_rate=0.80)
        
    solution = max(ea.population)    
    best_weight = sum([items[i][0] for i in range(len(solution.candidate)) if solution.candidate[i] == 1])

    best = [max(0, solution.fitness), best_weight, solution.candidate]
    return best

In [ ]:
def generation_analysis(cost_total, relevance_total, items, solver, iterations):
    generations = range(10, 110, 10)
    header = 'Budget & ' + ' & '.join([str(x) for x in generations]) + r"\\"
    print(header)
    
    for budget_factor in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        intermediate = []
        for gen in generations:
            global MAXGEN
            MAXGEN = gen
            capacity = budget_factor * cost_total
            avg = []
            for i in range(iterations):
                best = solver(items, capacity)
                avg.append(best[0])
            intermediate.append(str.format('{:{prec}}', sum(avg)/len(avg), prec='.4'))
        row = str(budget_factor) + ' & ' + ' & '.join(intermediate) + r"\\"
        print(row)

In [ ]:
def run_experiment(cost_total, relevance_total, items, solver, iterations=1):
    print('BudgetFactor;ElapsedTime;Value;Weight;Solution')
    for budget_factor in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        capacity = budget_factor * cost_total
        for i in range(iterations):
            start = time()
            best = solver(items, capacity)
            end = time()

            print(str.format('{0};{1:0.3f};{2};{3};{4}', budget_factor, end - start, best[0], best[1], best[2]))

#### Synthetic queries

In [ ]:
MAXGEN = 60    

In [ ]:
num_queries = 25
items = get_synthetic_queries(num_queries)
cost_total = sum([x[0] for x in items])
relevance_total = sum([x[1] for x in items])
print('Max cumulative relevance: ', relevance_total)
print('Total cost: ', cost_total)

run_experiment(cost_total, relevance_total, items, genetic_algorithm_solver, iterations=30)
generation_analysis(cost_total, relevance_total, items, genetic_algorithm_solver, iterations=30)

In [ ]:
items = get_adult_dataset_queries()
print('Queries: ', items)

cost_total = sum([x[0] for x in items])
relevance_total = sum([x[1] for x in items])
print('Max cumulative relevance: ', relevance_total)
print('Total cost: ', cost_total)

run_experiment(cost_total, relevance_total, items, genetic_algorithm_solver, iterations=30)
generation_analysis(cost_total, relevance_total, items, genetic_algorithm_solver, iterations=30)